In [ ]:
from pathlib import Path

In [ ]:
dataset_path = Path.cwd().joinpath("datasets", "halifax_site")

In [ ]:
import pandas as pd

In [ ]:
training_data = pd.read_csv(
    dataset_path.joinpath("training_data.csv"), sep="|",  lineterminator='\n')

In [ ]:
test_data = pd.read_csv(dataset_path.joinpath(
    "test_data.csv"), sep="|",  lineterminator='\n')

In [ ]:
training_data.head()

In [ ]:
training_data = training_data[~training_data.content.isna()
                           & ~training_data.url.isna()]

In [ ]:
test_data = test_data[~test_data.content.isna() & ~test_data.url.isna()]

In [ ]:
test_data.head()

In [ ]:
test_data.shape

In [ ]:
from haystack import Document

In [ ]:
train_documents = [
    Document(content=str(example.content_cleaned), meta={"url": example.url}) for example in training_data.itertuples()
]

In [ ]:
test_documents = [
    Document(content=str(example.content_cleaned), meta={"url": example.url}) for example in test_data.itertuples()
]

In [ ]:
all_documents = train_documents + test_documents

In [ ]:
embedding_model_id = "dunzhang/stella_en_400M_v5"

In [ ]:
### need to come back here, and download the stuff.

from haystack import Pipeline

In [ ]:
from haystack import Pipeline

In [ ]:
indexing_pipeline = Pipeline()

In [ ]:
all_documents[0]

In [ ]:
from haystack.components.writers import DocumentWriter

In [ ]:
from haystack_integrations.document_stores.pgvector import PgvectorDocumentStore

In [ ]:
from src.shared.database import postgres_uri

In [ ]:
import os

In [ ]:
os.environ["PG_CONN_STR"] = postgres_uri

In [ ]:
document_store = PgvectorDocumentStore(
    embedding_dimension=1024,
    vector_function="cosine_similarity",
    recreate_table=True,
    search_strategy="hnsw",
)

In [ ]:
from haystack.document_stores.types import DuplicatePolicy

In [ ]:
document_writer = DocumentWriter(
    document_store=document_store, policy=DuplicatePolicy.OVERWRITE)

In [ ]:
from haystack.components.embedders import SentenceTransformersDocumentEmbedder

In [ ]:
embedder_component = SentenceTransformersDocumentEmbedder(
    model=embedding_model_id,
    normalize_embeddings=True,
    trust_remote_code=True,

)

In [ ]:
embedder_component.warm_up()

In [ ]:
indexing_pipeline.add_component(
    "embedder", embedder_component)
indexing_pipeline.add_component("writer", document_writer)


indexing_pipeline.connect("embedder", "writer")

In [ ]:
indexing_pipeline.run(data={"documents": all_documents[1:]})